<a href="https://colab.research.google.com/github/Ctc4/ATV-BIGDATA-SPARK/blob/main/COMECANDO_SPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRABALHANDO COM SPARK

In [2]:
!pip install -q findspark

In [3]:
import findspark

In [4]:
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>